# Gesture Recognition Using CNN with RNN
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
from numpy.random import randint

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('/mnt/disks/user/project/PROJECT/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/mnt/disks/user/project/PROJECT/Project_data/val.csv').readlines())
batch_size = 80
idx_size = 30

In [4]:
cwd = os.getcwd()
cwd

'/mnt/disks/user/project/PROJECT'

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    np.random.seed(5)
#     img_idx = [x for x in randint(0,30,idx_size)] #create a list of image numbers you want to use for a particular video
    img_idx = [x for x in range(idx_size)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            
            batch_data = np.zeros((batch_size,len(img_idx),110,110,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            for folder in range(batch_size): # iterate over the batch_size
                
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape                    
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    image = imresize(image, (110, 110))
                    
                    batch_data[folder,idx,:,:,0] = image[:, : , 0] - np.min(image[:, : , 0])/np.max(image[:, : , 0]) - np.min(image[:, : , 0])
                    batch_data[folder,idx,:,:,1] = image[:, : , 1] - np.min(image[:, : , 1])/np.max(image[:, : , 1]) - np.min(image[:, : , 1])
                    batch_data[folder,idx,:,:,2] = image[:, : , 2] - np.min(image[:, : , 2])/np.max(image[:, : , 2]) - np.min(image[:, : , 2])
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
         # write the code for the remaining data points which are left after full batches
        rest_batch = len(folder_list)%batch_size
        batch_data = np.zeros((rest_batch,idx_size,110,110,3)) 
        batch_labels = np.zeros((rest_batch,5))
        for folder in range(rest_batch): # iterate over the batch_size
            imgs = os.listdir(source_path+'/'+ t[folder + (len(folder_list) - rest_batch)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = imread(source_path+'/'+ t[folder + (len(folder_list) - rest_batch)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                image = imresize(image, (110, 110))

                batch_data[folder,idx,:,:,0] = image[:, : , 0] - np.min(image[:, : , 0])/np.max(image[:, : , 0]) - np.min(image[:, : , 0])
                batch_data[folder,idx,:,:,1] = image[:, : , 1] - np.min(image[:, : , 1])/np.max(image[:, : , 1]) - np.min(image[:, : , 1])
                batch_data[folder,idx,:,:,2] = image[:, : , 2] - np.min(image[:, : , 2])/np.max(image[:, : , 2]) - np.min(image[:, : , 2])

            batch_labels[folder, int(t[folder + (len(folder_list) - rest_batch)].strip().split(';')[2])] = 1
        yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import LSTM, SimpleRNN

#write your model here

rnn_model = Sequential()
rnn_model.add(TimeDistributed(Conv2D(16,kernel_size=(3,3), activation ="relu"),input_shape=(idx_size,110, 110, 3)))
rnn_model.add(TimeDistributed(BatchNormalization()))
rnn_model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))
rnn_model.add(TimeDistributed(Dropout(0.25)))

rnn_model.add(TimeDistributed(Conv2D(16,kernel_size=(3,3), activation ="relu")))
rnn_model.add(TimeDistributed(BatchNormalization()))
rnn_model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))
rnn_model.add(TimeDistributed(Dropout(0.25)))

rnn_model.add(TimeDistributed(Conv2D(32,kernel_size=(3,3), activation ="relu")))
rnn_model.add(TimeDistributed(BatchNormalization()))
rnn_model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))
rnn_model.add(TimeDistributed(Dropout(0.25)))

# rnn_model.add(TimeDistributed(Conv2D(64,kernel_size=(3,3), activation ="relu")))
# rnn_model.add(TimeDistributed(BatchNormalization()))
# rnn_model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))
# rnn_model.add(TimeDistributed(Dropout(0.25)))

# rnn_model.add(TimeDistributed(Conv2D(128,kernel_size=(3,3), activation ="relu")))
# rnn_model.add(TimeDistributed(BatchNormalization()))
# rnn_model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))
# rnn_model.add(TimeDistributed(Dropout(0.25)))

rnn_model.add(TimeDistributed(Flatten()))
rnn_model.add(LSTM(32, activation = "tanh"))
rnn_model.add(Dense(256,activation='relu'))
rnn_model.add(Dropout(0.25))
rnn_model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = 'Adam'#write your optimizer
rnn_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (rnn_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 108, 108, 16)  448       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 108, 108, 16)  64        
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 54, 54, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 54, 54, 16)    0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 52, 52, 16)    2320      
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 52, 52, 16)    64        
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 26, 26, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
rnn_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  /mnt/disks/user/project/PROJECT/Project_data/train ; batch size = 80
 /mnt/disks/user/project/PROJECT/Project_data/val ; batch size = 80
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


9/9 [==============================] - 100s 11s/step - loss: 1.5699 - categorical_accuracy: 0.2792 - val_loss: 1.4543 - val_categorical_accuracy: 0.4300

Epoch 00001: saving model to model_init_2020-02-1015_02_28.504976/model-00001-1.57841-0.26395-1.45427-0.43000.h5
Epoch 2/10
9/9 [==============================] - 79s 9s/step - loss: 1.3790 - categorical_accuracy: 0.4486 - val_loss: 1.2606 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to model_init_2020-02-1015_02_28.504976/model-00002-1.37143-0.46305-1.26062-0.50000.h5
Epoch 3/10
9/9 [==============================] - 80s 9s/step - loss: 1.1735 - categorical_accuracy: 0.5525 - val_loss: 1.0910 - val_categorical_accuracy: 0.5900

Epoch 00003: saving model to model_init_2020-02-1015_02_28.504976/model-00003-1.17556-0.54148-1.09099-0.59000.h5
Epoch 4/10
9/9 [==============================] - 80s 9s/step - loss: 1.0172 - categorical_accuracy: 0.6107 - val_loss: 1.0737 - val_categorical_accuracy: 0.4900

Epoch 00004: savin